In [1]:
from __future__ import division
from collections import Counter
from collections import defaultdict

In [2]:
#Definindo Usuários
users = [
    { "id": 0, "name": "Hero" },
    { "id": 1, "name": "Dunn" },
    { "id": 2, "name": "Sue" },
    { "id": 3, "name": "Chi" },
    { "id": 4, "name": "Thor" },
    { "id": 5, "name": "Clive" },
    { "id": 6, "name": "Hicks" },
    { "id": 7, "name": "Devin" },
    { "id": 8, "name": "Kate" },
    { "id": 9, "name": "Klein" }
]

In [3]:
#Definindo Amigos
friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),(4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]

In [4]:
# adicionar uma lista de amigos para cada usuário
for user in users:
    user["friends"] = []

In [5]:
# popular a lista com os dados de friendships
for i, j in friendships:
    # isso funciona porque users[i] é o usuário cuja id é i
    users[i]["friends"].append(users[j]) # adiciona i como um amigo de j
    users[j]["friends"].append(users[i]) # adiciona j como um amigo de i

In [6]:
# encontramos um número total de conexões, resumindo os tamanhos de todas as listas de friends:
def number_of_friends(user):
    """quantos amigos o usuário tem?"""
    return len(user["friends"]) # tamanho da lista friend_ids

total_connections = sum(number_of_friends(user) for user in users) #24

In [7]:
#número médio de conexões
num_users = len(users) # tamanho da lista de usuários

avg_connections = total_connections / num_users # 2,4

In [8]:
# encontrar as pessoas mais conectadas

# cria uma lista (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]

# ordená-los de “muito amigos” para “menos amigos”
sorted(num_friends_by_id,
       key= lambda userid_numfriends: userid_numfriends[1],
       reverse=True)

# cada par é (user_id, num_friends)
# [(1, 3), (2, 3), (3, 3), (5, 3), (8, 3),
# (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]

[(1, 3),
 (2, 3),
 (3, 3),
 (5, 3),
 (8, 3),
 (0, 2),
 (4, 2),
 (6, 2),
 (7, 2),
 (9, 1)]

In [9]:
#sugerir um usuário que possa conhecer amigos de amigos

def friends_of_friend_ids_bad(user):
    # “foaf” é abreviação de “friend of a friend”
    return [foaf["id"] 
            for friend in user["friends"] # para cada amigo de usuário
            for foaf in friend["friends"]] # pega cada _their_friends

#print([friend["id"] for friend in users[0]["friends"]])
#print([friend["id"] for friend in users[1]["friends"]])
#print([friend["id"] for friend in users[2]["friends"]])

In [10]:
#excluir as pessoas que já são conhecidas do usuário

def not_the_same(user, other_user):
    """dois usuários não são os mesmos se possuem ids diferentes"""
    return user["id"] != other_user["id"]


def not_friends(user, other_user):
    """other_user não é um amigo se não está em user[“friends”];
    isso é, se é not_the_same com todas as pessoas em user[“friends”]"""
    return all(not_the_same(friend, other_user)
               for friend in user["friends"])

def friends_of_friend_ids(user):
    return Counter(foaf["id"]
                   for friend in user["friends"] # para cada um dos meus amigos
                   for foaf in friend["friends"] # que contam *their* amigos
                   if not_the_same(user, foaf) # que não sejam eu
                   and not_friends(user, foaf)) # e que não são meus amigos

In [ ]:
#print (friends_of_friend_ids(users[3]))
#Counter({0: 2, 5: 1})

# Isso diz sobre Chi (id 3) que ela possui dois amigos em comum com Hero (id 0) mas somente um amigo em comum com Clive (id 5)

In [11]:
# encontrar usuários com interesses similares

# lista de pares (user_id, interest)

interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [12]:
# função que encontre usuários com o mesmo interesse

def data_scientists_who_like(target_interest):
    return [user_id
            for user_id, user_interest in interests
            if user_interest == target_interest]

In [13]:
# Para melhorar a performance é melhor:

### 1 - construir um índice de interesses para usuários

# as chaves são interesses, os valores são listas de user_ids com interests
user_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)
    
### 2 - índice de usuários para interesses

# as chaves são user_ids, os valores são as listas de interests para aquele user_id
interests_by_user_id = defaultdict(list)

for user_id, interest in interests:
    interests_by_user_id[user_id].append(interest)

In [ ]:
# Agora fica fácil descobrir quem possui os maiores interesses em comum com um certo usuário:
# * Itera sobre os interesses do usuário.
# * Para cada interesse, itera sobre os outros usuários com aquele interesse.
# * Mantém a contagem de quantas vezes vemos cada outro usuário.


In [14]:
def most_common_interests_with(user):
    return Counter(interested_user_id
                   for interest in interests_by_user_id[user["id"]]
                   for interested_user_id in user_ids_by_interest[interest]
                   if interested_user_id != user["id"])

In [15]:
#Salários e Experiência

salaries_and_tenures = [(83000, 8.7), (88000, 8.1),
                        (48000, 0.7), (76000, 6),
                        (69000, 6.5), (76000, 7.5),
                        (60000, 2.5), (83000, 10),
                        (48000, 1.9), (63000, 4.2)]

In [16]:
### analisar a média salarial para cada ano
# as chaves são os anos, os valores são as listas dos salários para cada ano
salary_by_tenure = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    salary_by_tenure[tenure].append(salary)

# as chaves são os anos, cada valor é a média salarial para aquele ano
average_salary_by_tenure = {tenure : sum(salaries) / len(salaries)
                            for tenure, salaries in salary_by_tenure.items()
                           }

In [17]:
#já que nenhum dos usuários possui o mesmo caso, Talvez fosse mais proveitoso agrupar os casos

def tenure_bucket(tenure):
    if tenure < 2:
        return "less than two"
    elif tenure < 5:
        return "between two and five"
    else:
        return "more than five"
    
#o grupo junta os salários correspondentes para cada agrupamento
# as chaves são agrupamentos dos casos, os valores são as listas
# dos salários para aquele agrupamento
salary_by_tenure_bucket = defaultdict(list)

for salary, tenure in salaries_and_tenures:
    bucket = tenure_bucket(tenure)
    salary_by_tenure_bucket[bucket].append(salary)
    
# as chaves são agrupamentos dos casos, os valores são
# a média salarial para aquele agrupamento
average_salary_by_bucket = {
    tenure_bucket : sum(salaries) / len(salaries)
    for tenure_bucket, salaries in salary_by_tenure_bucket.items()
}

In [20]:
#Contas Pagas
# criar um modelo prever “paid” para os usuários com poucos e muitos anos de experiência,
#e “unpaid” para os usuários com quantidade mediana de experiência

def predict_paid_or_unpaid(years_experience):
    if years_experience < 3.0:
        return "paid"
    elif years_experience < 8.5:
        return "unpaid"
    else:
        return "paid"

In [19]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0, "HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"),
    (1, "Postgres"), (2, "Python"), (2, "scikit-learn"), (2, "scipy"),
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Python"),
    (3, "statistics"), (3, "regression"), (3, "probability"),
    (4, "machine learning"), (4, "regression"), (4, "decision trees"),
    (4, "libsvm"), (5, "Python"), (5, "R"), (5, "Java"), (5, "C++"),
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"),
    (6, "probability"), (6, "mathematics"), (6, "theory"),
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"),
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"),
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"),
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

In [ ]:
# Uma simples forma (e também fascinante) de encontrar os interesses mais
# populares é fazer uma simples contagem de palavras:
# 1 - Coloque cada um em letras minúsculas (já que usuários diferentes podem
#      ou não escrever seus interesses em letras maiúsculas).
# 2 - Divida em palavras.
# 3 - Conte os resultados.

In [ ]:
words_and_counts = Counter(word for user, interest in interests
                           for word in interest.lower().split())


In [ ]:
# listar as palavras que ocorrem mais de uma vez
for word, count in words_and_counts.most_common():
    if count > 1:
        print(word, count)